In [1]:
%run "../Functions/4. User comparison.ipynb"

C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (18,21,22,28,32,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (14,15,16,17,18,20,21,22,25,26,27,30,33,34,35,36,37,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (18,21,22,28,32,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (14,15,16,17,18,20,21,22,25,26,27,30,33,34,35,36,37,

# Prepare data

## Prepare Google form data

In [2]:
# Rename columns of the Google Forms table with tags independantly of form language
columnTags = ["timestamp", "gameInterest", "gameFrequency", "age", "gender", "biologyStudy", "biologyInterest", "synthBioKnowledge", "biobrickKnowledge", "previousVersion", "previousPlay", "arcadePlay", "androidPlay", "Q1", "Q2", "Q3", "Q4", "Q5", "Q6", "Q7", "Q8", "Q9", "Q10", "Q11", "Q12", "Q13", "Q14", "Q15", "Q16", "Q17", "Q18", "Q19", "Q20", "Q21", "Q22", "Q23", "Q24", "Q25", "Q26", "Q27", "comments", "anonymousID"]
columnQuestions = gform.columns.values.tolist()
googleData = gform.rename(columns=dict(zip(columnQuestions, columnTags)))
#googleData.head()

In [3]:
# Replaces answers to scientific questions in the questionnaires by their values (True or False)
correctedData = googleData
for rowId in range(correctedData.shape[0]):
    # Get the correction for each subject
    playerId = correctedData.loc[rowId, "anonymousID"]
    correction = getCorrections(playerId)
    if correction.shape[1] > 0:
        # If subject has answered questionnaire
        correction = correction.rename(index=dict(zip(columnQuestions, columnTags)))
        # Replace scientific answers by their correction
        for questionId in range(27):
            questionTag = "Q" + str(questionId + 1)
            correctedData.loc[rowId, questionTag] = int(correction.loc[questionTag, "corrections" + str(rowId)])
    
#correctedData.head()

In [8]:
list(zip(columnQuestions, columnTags))

[('Timestamp', 'timestamp'),
 ('Are you interested in video games?', 'gameInterest'),
 ('Do you play video games?', 'gameFrequency'),
 ('How old are you?', 'age'),
 ('What is your gender?', 'gender'),
 ('How long have you studied biology?', 'biologyStudy'),
 ('Are you interested in biology?', 'biologyInterest'),
 ('Before playing Hero.Coli, had you ever heard about synthetic biology?',
  'synthBioKnowledge'),
 ('Before playing Hero.Coli, had you ever heard about BioBricks?',
  'biobrickKnowledge'),
 ('Have you ever played an older version of Hero.Coli before?',
  'previousVersion'),
 ('Have you played the current version of Hero.Coli?', 'previousPlay'),
 ('Have you played the arcade cabinet version of Hero.Coli?', 'arcadePlay'),
 ('Have you played the Android version of Hero.Coli?', 'androidPlay'),
 ('In order to modify the abilities of the bacterium, you have to...', 'Q1'),
 ('What are BioBricks and devices?', 'Q2'),
 ('What is the name of this BioBrick?', 'Q3'),
 ('What is the name o

## For association rule mining

In [4]:
# Get only answers to scientific questions
correctedScientific = correctedData.loc[:, "Q1":"Q27"]
#correctedScientific.head()

In [5]:
# Save database in file
correctedScientific.to_csv("dataForOrange/correctedScientific.csv")